# Basics of sequence to sequence (seq2seq) models

## Character level translation model

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Note: a lot of variable names are shorthands (i.e seqlen = sequence length)
Rationale being I don't want essays as variable names

feel free to change ^

In [1]:
# imports

import numpy as np
import keras

### Download the dataset [here](http://www.manythings.org/anki/fra-eng.zip)

In [2]:
#i have no ieda if this eve nwroks

# fpath = keras.utils.get_file(origin="http://www.manythings.org/anki/fra-eng.zip")
# dirpath = Path(fpath).parent.absolute()
# os.system(f"unzip -q {fpath} -d {dirpath}")

### Config

In [11]:
batch_sz = 64
epoch = 100
latent_dim = 256
num_samples = 10000

data_path = "/content/drive/MyDrive/GenAI/fra.txt"

### Preprocessing Data

In [12]:
input_texts, target_texts = [], []
input_chars, target_chars = set(), set()

with open(data_path, "r", encoding="utf-8") as file:
    lines = file.read().split("\n")

# populate working data
for line in lines[:min(num_samples, len(lines) - 1)]:
    # we use _ to indicate that a variable should be ignored
    # in this case, the 3rd column is the citation (something something),
    # which is irrelevant to our task at hand
    input_text, target_text, _ = line.split("\t")

    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)

    input_chars.update(input_text)
    target_chars.update(target_text)


input_chars, target_chars = sorted(list(input_chars)), sorted(list(target_chars))
num_enc_token, num_dec_token = len(input_chars), len(target_chars)
max_enc_seqlen, max_dec_seqlen = len(max(input_texts, key=len)), \
                                    len(max(target_texts, key=len))

print(f"Number of samples: {len(input_texts)}\n" +
      f"Number of unique input tokens: {num_enc_token}\n" +
      f"Number of unique output tokens: {num_dec_token}\n" +
      f"Max seqlen for input: {max_enc_seqlen}\n" +
      f"Max seqlen for output: {max_dec_seqlen}")

input_token_index, target_token_index = dict([(char, i) for i, char in enumerate(input_chars)]), \
                                        dict([(char, i) for i, char in enumerate(target_chars)])

encoder_input_data = np.zeros((len(input_texts),
                            max_enc_seqlen,
                            num_enc_token))

decoder_input_data = np.zeros((len(input_texts),
                                max_dec_seqlen,
                                num_dec_token))

decoder_target_data = np.zeros((len(input_texts),
                                max_dec_seqlen,
                                num_dec_token))

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max seqlen for input: 14
Max seqlen for output: 59


### Building the model

In [13]:
encoder_inputs = keras.Input(shape=(None, num_enc_token))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]
decoder_inputs = keras.Input(shape=(None, num_dec_token))

decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_dec_token, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Training

In [14]:
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_sz,
    epochs=epoch,
    validation_split=0.2,
)

#model.save("s2s_model.keras")

Epoch 1/100
125/125 [==============================] - 7s 22ms/step - loss: 1.1970 - accuracy: 0.7338 - val_loss: 1.0829 - val_accuracy: 0.7129
Epoch 2/100
125/125 [==============================] - 1s 10ms/step - loss: 0.9293 - accuracy: 0.7486 - val_loss: 0.9708 - val_accuracy: 0.7177
Epoch 3/100
125/125 [==============================] - 1s 10ms/step - loss: 0.8380 - accuracy: 0.7668 - val_loss: 0.8719 - val_accuracy: 0.7504
Epoch 4/100
125/125 [==============================] - 1s 10ms/step - loss: 0.7334 - accuracy: 0.7929 - val_loss: 0.7662 - val_accuracy: 0.7760
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6613 - accuracy: 0.8100 - val_loss: 0.7059 - val_accuracy: 0.7947
Epoch 6/100
125/125 [==============================] - 1s 10ms/step - loss: 0.6220 - accuracy: 0.8187 - val_loss: 0.6831 - val_accuracy: 0.8028
Epoch 7/100
125/125 [==============================] - 1s 12ms/step - loss: 0.5938 - accuracy: 0.8259 - val_loss: 0.6552 - val_accuracy:

In [15]:
import os
directory = '/content/drive/MyDrive/GenAI'
# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Save the model to the "gen_ai" directory
model.save(os.path.join(directory, "s2s_model.keras"))

### Performing inference

In [17]:
# load the model

model = keras.models.load_model(os.path.join(directory, "s2s_model.keras"))

### Create encoder/decoder

In [18]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [19]:
def decode_sequence(input_seq):
    # encode the input as state vectors
    states_value = encoder_model.predict(input_seq, verbose=0)

    # generate empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_dec_token))
    # populate the first character of target sequence with the start character
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value, verbose=0
        )

        # sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # exit condition: either hit max length or find stop character
        if sampled_char == "\n" or len(decoded_sentence) > max_dec_seqlen:
            stop_condition = True

        # update target seq
        target_seq = np.zeros((1, 1, num_dec_token))
        target_seq[0, 0, sampled_token_index] = 1.0

        # update states
        states_value = [h, c]
    return decoded_sentence

### Generating sentences

In [20]:
for seq_index in range(20):
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Cassez-vous.

-
Input sentence: Go.
Decoded sentence: Cassez-vous.

-
Input sentence: Go.
Decoded sentence: Cassez-vous.

-
Input sentence: Go.
Decoded sentence: Cassez-vous.

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sente